# Simulating a Low-Pass Filter in a Circuit

Copyright 2024 Mike Augspurger, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)), based on an idea by Allen Downey

## Introduction

In experimentation, we often need to record a *dynamic signal*: that is, a signal that changes with time, like the vibration of a bridge, the velocity of wind, the pressure in a speaker.  These signals are often *complex*: they contain multiple dynamic signals all added together to create a complicated data set.   One way of simplifying such a data set is to *filter* the signal: if we know that the high frequency component of an input signal is some kind of unwanted "noise", we can take those high frequency components out using a *low pass filter*.

<br>

In this notebook, we will simulate the low-pass filter for input signals to determine how well they remove unwanted noise.  The input signals will be sinusoidal waves of this form:

<br>

$$ V_{in}(t) = A \cos (2 \pi f t) $$

<br>

where $A$ is the amplitude of the input signal (usually in units of volts), and $f$ is the frequency of the signal in Hz.   The frequency of a signal tells you how many times the signal oscillates in a second.  Here, for instance, is a time plot of a 10 Hz wave with an amplitude of 5 V:

<br><center>

<img src=https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/sinusoidal_wave.png width=500></center>

<br>

Notice that it oscillates 10 times in 1 second (count the peaks!) and has a amplitude that ranges from -5 to 5 V.

## Part 1: Setting up a system and state

The resistance and capacitance of the components in the circuit define the behavior of the circuit in response to changes. The amplitude and frequency of the input signal define the input values for the filter.   We want to put these values in a system dictionary.  But in doing so, we also want to calculate a couple parameters that are derive from these values:

* `omega` is the frequency of the input signal in radians/second.  We can use this value in numpy functions.  To calculate `omega`:

$$\omega = 2\pi f$$

<br>

* `tau` is the time constant for this circuit, which is roughly the time it takes to charge (i.e. fill) the capacitor.  As we saw with the differential equation, `tau` is equal to:

$$\tau = \frac{1}{RC}$$

<br>

* Finally, we want to end the simulation after 4 cycles of the input signal.  So define `t_end` as the time required to do that (think about how long it takes to complete one cycle for a signal with a given frequency; note that the units $Hz$ is equivalent to $\frac{1}{s}$.

Adapt the `make_system` and `make_state` from the coffee notebooks below to our new purpose:

In [ ]:
import pandas as pd
import numpy as np

# Define parameters
R1 = 1.0e6   # ohm
C1 = 1.0e-9  # farad
A = 5.0      # volt
f = 1000.0   # Hz

def make_system(R1,C1,A,f):
    # Calculate tau, omega, and t_end in here
    # dt should be much smaller than t_end: you can calculate that here too
    return dict(T_init=T_init, T_final=0, volume=volume,
                  r=r, t_end=t_end, T_env=T_env, T_goal=T_goal,
                  t_0=0,  dt=1)

Now make a state by adapting the coffee code again.  What is the state variable?  It makes sense to give the state variable an initial value of 0:

In [ ]:
def make_state(system):
    return pd.Series(dict(temp=system['T_init']), name='State object')

Finally, call the two functions and make sure their output makes sense:

In [ ]:
system1 = make_system(R1,C1,A,f)
state1 = make_state(system1)
print(system1)
print(state1)

## Part 2: The change function

You'll need a change function that incorporates the differential equation from Notebook 2.4.7.  You'll have to calculate $V_{in}$ using the equation at the top of this notebook.  `Omega` should come in handy!

<br>

As usual, the function should return the change in the state variable for that time step.




In [ ]:
# Here's the change_func from the coffee code
def change_func(t, state, system):
    r, T_env, dt = system['r'], system['T_env'], system['dt']
    deltaT =  -r * (state.temp - T_env) * dt
    return deltaT

In [ ]:
# Now test your change function: the return value should
# change depending on the value of t that you enter!
dT_coffee = change_func(0,state,system)
print(dT_coffee)

## Part 3:  Run_simulation


Make adjustments to run simulation as necessary.  The tricky change here is that you'll want to print both $V_{in}$ and $V_{out}$.  You'll find $V_{out}$ in the `for` loop, but you can calculate $V_{in}$ directly using `t_array`.  Look back at the "Historical Population" exercise from 2 weeks back to remember how to do that!

In [ ]:
def run_simulation(system, change_func):
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)
    state = make_state(system)
    results = pd.Series(index=t_array,dtype=object)
    results.iloc[0] = state.temp

    for i in range(n-1):
        t = t_array[i]
        delta = change_func(t, state, system)
        state.temp = state.temp + delta
        results.iloc[i+1] = state.temp

    system['T_final'] = results.iloc[-1]
    return results

In [ ]:
# Plot your results and make sure they make sense

results.plot(xlabel='time (s)', ylabel='Temperature (degrees C',
                 title='Temperature of Cooling Object', label='Aluminum', legend=True)


If things have gone according to plan, the amplitude of the output signal should be about 0.8 V.

## Sweeping frequency

Here's what `V_mid` looks like for a range of frequencies:

In [ ]:
from matplotlib.pyplot import subplot

fs = [1, 10, 100, 1000, 10000, 100000]

for i in range(6):
    R1 = 1.0e6
    C1 = 1.0e-9
    A = 5.0
    f = fs[i]
    params = R1, C1, A, f
    system = make_system(params)
    results, details = run_solve_ivp(system, slope_func)
    subplot(3, 2, i+1)
    plot_results(results)

The low frequencies start in the upper right corner.  Notice the amplitudes, which are the same for $f = 1$ and $f = 10$.  For higher frequencies, the signal is mostly filtered out.  Notice that the cutoff frequency for this circuit (as defined in make_system above) is 159 Hz.  Do you see what that indicates here?

## Estimating the output ratio

Let's compare the amplitudes of the input and output signals.  Below the cutoff frequency, we expect them to be about the same.  Above the cutoff, we expect the amplitude of the output signal to be smaller.

We'll start with a signal at the cutoff frequency, `f=1000` Hz.

In [ ]:
R1 = 1.0e6
C1 = 1.0e-9
A = 5.0
f = 1000
params = R1, C1, A, f
system = make_system(params)
results, details = run_solve_ivp(system, slope_func)
V_mid = results.V_mid
plot_results(results)

The following function computes `V_in` as a `TimeSeries`:

In [ ]:
def compute_vin(results, system):
    """Computes V_in as a TimeSeries.

    results: TimeFrame with simulation results
    system: System object with A and omega

    returns: TimeSeries
    """
    A, omega = system['A'], system['omega']

    ts = results.index
    V_in = A * np.cos(omega * ts)
    return pd.Series(data=V_in, index=results.index, name='V_in')

Here's what the input and output look like.  Notice that the output is not just smaller; it is also "out of phase"; that is, the peaks of the output are shifted to the right, relative to the peaks of the input.

In [ ]:
V_in = compute_vin(results, system)

V_mid.plot()
V_in.plot(xlabel='Time (s)',
         ylabel='V (volt)');

The following function estimates the amplitude of a signal by computing half the distance between the min and max.

In [ ]:
def estimate_A(series):
    """Estimate amplitude.

    series: TimeSeries

    returns: amplitude in volts
    """
    return (series.max() - series.min()) / 2

The amplitude of `V_in` should be near 5 (but not exact because we evaluated it at a finite number of points).

In [ ]:
A_in = estimate_A(V_in)
A_in

The amplitude of `V_mid` should be lower.

In [ ]:
A_out = estimate_A(V_mid)
A_out

And here's the ratio between them.

In [ ]:
ratio = A_out / A_in
ratio

### Exercise 2

Encapsulate the code we have so far in a function that takes two `Series` objects and returns the ratio between their amplitudes.

In [ ]:
# Define the function estimate_ratio
def estimate_ratio(series1,series2):
    return estimate_A(series1)/estimate_A(series2)

And test your function.

In [ ]:
estimate_ratio(V_mid, V_in)

### Exercise 3

Write a function that takes as parameters an array of input frequencies as well as the parameters for a filter.

For each input frequency it should run a simulation and use the results to estimate the output ratio (dimensionless).

It should return a `Series` object with the ratio for each input frequency.

In [ ]:
# Define function sweep_frequency

def sweep_frequency(fs,params):
    sweep_output = pd.Series([],dtype=np.float64)
    R1, C1, A, f = params
    for f_var in fs:
        params =  R1, C1, A, f_var
        system = make_system(params)
        results, details = run_solve_ivp(system, slope_func)
        V_mid = results.V_mid
        V_in = compute_vin(results, system)
        sweep_output[f_var]=estimate_ratio(V_mid,V_in)

    return sweep_output

Run your function with these frequencies.

In [ ]:
fs = 10 ** linspace(0, 4, 9)

In [ ]:
R1 = 1.0e6
C1 = 1.0e-9
A = 5.0
f = 1000.0

ratios = sweep_frequency(fs, params)
pd.DataFrame(ratios)

We can plot output ratios like this:

In [ ]:
ratios.plot(color='C2', label='output ratio',xlabel='Frequency (Hz)',
         ylabel='$V_{out} / V_{in}$',legend=True);

But it is useful and conventional to plot ratios on a log-log scale.  The vertical gray line shows the cutoff frequency.

In [ ]:
def plot_ratios(ratios, system):
    """Plot output ratios.
    """

    cutoff = magnitude(system['cutoff'])
    plt.axvline(cutoff, color='gray', alpha=0.4)

    ratios.plot(color='C2', label='output ratio',
                xlabel='Frequency (Hz)',
                ylabel='$V_{out} / V_{in}$',
                logx=True,logy=True,legend=True)

In [ ]:
plot_ratios(ratios, system)

This plot shows the cutoff behavior more clearly.  To the left of the cutoff (lower frequencies), the output ratio is close to 1.  To the right of the cutoff, it drops off linearly on a log scale, which indicates that output ratios for high frequencies are practically 0.

### Exercise 4

By analysis we can show that the output ratio for this signal is

$A = \frac{1}{\sqrt{1 + (R C \omega)^2}}$

where $\omega = 2 \pi f$.

Write a function that takes an array of input frequencies (and parameters) and returns $A(f)$ as `SweepSeries` objects.  Plot the object and compare it with the results from the previous section.


In [ ]:
# Define the function output_ratios_analy

def output_ratios_analy(fs,params):
    """Computes analytically the amplitude ratio
    for an array of input frequencies fs.

    fs: Array of input frequencies
    params: parameters of the filter (R1, C1, A, f)

    returns: SweepSeries
    """
    R1, C1, A, f = params
    sweep_output = pd.Series([],dtype=np.float64)

    for f_var in fs:
        omega = 2 * np.pi * f_var
        radical = 1 + (R1 * C1 * omega)**2
        ratio = 1/np.sqrt(radical)
        sweep_output[f_var] = ratio

    return sweep_output

Test your function:

In [ ]:
fs = 10 ** linspace(0, 4, 9)
A = output_ratios_analy(fs,params)

Plot the theoretical results along with the simulation results and see if they agree.

In [ ]:
A.plot(style=':', color='gray', label='analysis',legend=True)
plot_ratios(ratios, system)